In [67]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd


In [68]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

--2024-03-23 18:19:33--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.207, 172.217.194.207, 142.250.4.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip.2’

10_food_classes_all 100%[===================>] 495.13M  23.3MB/s    in 23s     

2024-03-23 18:19:56 (21.9 MB/s) - ‘10_food_classes_all_data.zip.2’ saved [519183241/519183241]



In [69]:
import zipfile
import os
zipref=zipfile.ZipFile("10_food_classes_all_data.zip")
zipref.extractall()
zipref.close()

In [70]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)
train_datagen= ImageDataGenerator()
test_datagen= ImageDataGenerator()
train_data=train_datagen.flow_from_directory('10_food_classes_all_data/train/',
                                  batch_size=32,
                                  target_size=(128,128),
                                  class_mode='categorical',
                                  seed=42,
                                  shuffle=True)
test_data= test_datagen.flow_from_directory('10_food_classes_all_data/test/',
                                 batch_size=32,
                                 target_size=(128,128),
                                 class_mode='categorical',
                                 seed=42,
                                 shuffle=True)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [71]:
base_model=tf.keras.applications.VGG19(include_top=False)
base_model.trainable=False
inputs= tf.keras.layers.Input(shape=(128,128,3))
x= base_model(inputs)
x= tf.keras.layers.GlobalAveragePooling2D()(x)
x= tf.keras.layers.Flatten()(x)
output= tf.keras.layers.Dense(10, activation='softmax')(x)
model= tf.keras.models.Model(inputs,output)
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d_9  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 flatten_9 (Flatten)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 10)                5130      
                                                                 
Total params: 20029514 (76.41 MB)
Trainable params: 5130 (20.04 KB)
Non-trainable params: 20024384 (76.39 MB)
_______________

The VGG (Visual Geometry Group) models are known for their simplicity and effectiveness. Variants include VGG16 and VGG19, which consist of multiple convolutional layers followed by fully connected layers.

In [72]:
model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(train_data, epochs=5,
          validation_data=test_data,
          steps_per_epoch=len(train_data),
          validation_steps=int(0.2*len(test_data)))

Epoch 1/5
235/235 [==============================] - 23s 93ms/step - loss: 5.7288 - accuracy: 0.3521 - val_loss: 3.0619 - val_accuracy: 0.5042
Epoch 2/5
235/235 [==============================] - 21s 88ms/step - loss: 2.9351 - accuracy: 0.5213 - val_loss: 2.7273 - val_accuracy: 0.5042
Epoch 3/5
235/235 [==============================] - 22s 92ms/step - loss: 2.1867 - accuracy: 0.5788 - val_loss: 2.1702 - val_accuracy: 0.5792
Epoch 4/5
235/235 [==============================] - 21s 88ms/step - loss: 1.7740 - accuracy: 0.6147 - val_loss: 1.9826 - val_accuracy: 0.5792
Epoch 5/5
235/235 [==============================] - 21s 91ms/step - loss: 1.5123 - accuracy: 0.6367 - val_loss: 1.9237 - val_accuracy: 0.5500


In [73]:
model.trainable

True

In [74]:
len(base_model.layers)

22

In [75]:
for num, layer in enumerate(base_model.layers):
  print(num, layer.name, layer.trainable)

0 input_21 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 False
18 block5_conv2 False
19 block5_conv3 False
20 block5_conv4 False
21 block5_pool False


In [76]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [77]:
base_model.trainable= True
for i ,layer in enumerate(base_model.layers[:-5]):
  layer.trainable=False

opening the last 5 threads of the model to put it into action for better result

In [78]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name, layer.trainable)

0 input_21 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True


In [79]:
model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
model.fit(train_data, epochs=10,
          validation_data=test_data,
          steps_per_epoch=len(train_data),
          validation_steps=int(0.2*len(test_data)),
          initial_epoch=5)

Epoch 6/10
235/235 [==============================] - 25s 97ms/step - loss: 1.1955 - accuracy: 0.6281 - val_loss: 0.9222 - val_accuracy: 0.6979
Epoch 7/10
235/235 [==============================] - 22s 92ms/step - loss: 0.5654 - accuracy: 0.8167 - val_loss: 0.8398 - val_accuracy: 0.7208
Epoch 8/10
235/235 [==============================] - 23s 96ms/step - loss: 0.2595 - accuracy: 0.9159 - val_loss: 0.9882 - val_accuracy: 0.7229
Epoch 9/10
235/235 [==============================] - 23s 99ms/step - loss: 0.1462 - accuracy: 0.9537 - val_loss: 1.1892 - val_accuracy: 0.7271
Epoch 10/10
235/235 [==============================] - 22s 92ms/step - loss: 0.0916 - accuracy: 0.9721 - val_loss: 0.8523 - val_accuracy: 0.7833


reult after opening last 5 threads

In [80]:
model.evaluate(test_data)

79/79 [==============================] - 8s 95ms/step - loss: 0.9489 - accuracy: 0.7740


[0.9488765597343445, 0.7739999890327454]

model 2


In [95]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)
train_datagen= ImageDataGenerator()
test_datagen= ImageDataGenerator()
train_data=train_datagen.flow_from_directory('10_food_classes_all_data/train/',
                                  batch_size=32,
                                  target_size=(128,128),
                                  class_mode='categorical',
                                  seed=42,
                                  shuffle=True)
test_data= test_datagen.flow_from_directory('10_food_classes_all_data/test/',
                                 batch_size=32,
                                 target_size=(128,128),
                                 class_mode='categorical',
                                 seed=42,
                                 shuffle=True)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [96]:
base_model2=tf.keras.applications.DenseNet121(include_top=False)
base_model2.trainable=False
inputs= tf.keras.layers.Input(shape=(128,128,3))
x= base_model2(inputs)
x= tf.keras.layers.GlobalAveragePooling2D()(x)
x= tf.keras.layers.Flatten()(x)
output= tf.keras.layers.Dense(10, activation='softmax')(x)
model= tf.keras.models.Model(inputs,output)
model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_32 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, None, None, 1024   7037504   
                             )                                   
                                                                 
 global_average_pooling2d_1  (None, 1024)              0         
 4 (GlobalAveragePooling2D)                                      
                                                                 
 flatten_14 (Flatten)        (None, 1024)              0         
                                                                 
 dense_14 (Dense)            (None, 10)                10250     
                                                                 
Total params: 7047754 (26.89 MB)
Trainable params: 10250 (

Using DenseNet121 DenseNet: DenseNet (Densely Connected Convolutional Networks) connects each layer to every other layer in a feed-forward fashion. This encourages feature reuse and facilitates gradient flow, leading to better parameter efficiency.

In [97]:
model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(train_data, epochs=5,
          validation_data=test_data,
          steps_per_epoch=len(train_data),
          validation_steps=int(0.2*len(test_data)))

Epoch 1/5
235/235 [==============================] - 31s 101ms/step - loss: 3.7572 - accuracy: 0.2405 - val_loss: 2.3465 - val_accuracy: 0.3500
Epoch 2/5
235/235 [==============================] - 23s 96ms/step - loss: 2.2872 - accuracy: 0.3865 - val_loss: 2.0845 - val_accuracy: 0.4208
Epoch 3/5
235/235 [==============================] - 23s 98ms/step - loss: 1.9410 - accuracy: 0.4431 - val_loss: 1.9549 - val_accuracy: 0.4688
Epoch 4/5
235/235 [==============================] - 20s 87ms/step - loss: 1.7697 - accuracy: 0.4741 - val_loss: 1.6819 - val_accuracy: 0.5000
Epoch 5/5
235/235 [==============================] - 22s 94ms/step - loss: 1.6403 - accuracy: 0.5009 - val_loss: 1.8644 - val_accuracy: 0.4792


In [98]:
len(base_model2.layers)

427

In [99]:
for num, layer in enumerate(base_model.layers):
  print(num, layer.name, layer.trainable)

0 input_21 True
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_conv4 True
11 block3_pool True
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True


In [100]:
base_model2.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_31 (InputLayer)       [(None, None, None, 3)]      0         []                            
                                                                                                  
 zero_padding2d_10 (ZeroPad  (None, None, None, 3)        0         ['input_31[0][0]']            
 ding2D)                                                                                          
                                                                                                  
 conv1/conv (Conv2D)         (None, None, None, 64)       9408      ['zero_padding2d_10[0][0]']   
                                                                                                  
 conv1/bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1/conv[0][0]']

In [101]:
base_model.trainable= True
for i ,layer in enumerate(base_model.layers[:-100]):
  layer.trainable=False

opening last 100 layers of the model casue model has many layers and most of em is relu so accuracy wont be much

In [102]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name, layer.trainable)

0 input_21 True
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_conv4 True
11 block3_pool True
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True


In [103]:
model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
model.fit(train_data, epochs=10,
          validation_data=test_data,
          steps_per_epoch=len(train_data),
          validation_steps=int(0.2*len(test_data)),
          initial_epoch=5)

Epoch 6/10
235/235 [==============================] - 63s 233ms/step - loss: 1.3463 - accuracy: 0.5656 - val_loss: 1.5750 - val_accuracy: 0.4667
Epoch 7/10
235/235 [==============================] - 22s 94ms/step - loss: 1.3182 - accuracy: 0.5689 - val_loss: 1.7079 - val_accuracy: 0.4625
Epoch 8/10
235/235 [==============================] - 21s 88ms/step - loss: 1.3060 - accuracy: 0.5760 - val_loss: 1.6126 - val_accuracy: 0.4875
Epoch 9/10
235/235 [==============================] - 23s 100ms/step - loss: 1.2934 - accuracy: 0.5787 - val_loss: 1.6061 - val_accuracy: 0.5146
Epoch 10/10
235/235 [==============================] - 23s 98ms/step - loss: 1.2879 - accuracy: 0.5811 - val_loss: 1.6871 - val_accuracy: 0.4771


In [104]:
model.evaluate(test_data)

79/79 [==============================] - 6s 79ms/step - loss: 1.5558 - accuracy: 0.5056


[1.5558314323425293, 0.5055999755859375]

model 3

In [105]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)
train_datagen= ImageDataGenerator()
test_datagen= ImageDataGenerator()
train_data=train_datagen.flow_from_directory('10_food_classes_all_data/train/',
                                  batch_size=32,
                                  target_size=(128,128),
                                  class_mode='categorical',
                                  seed=42,
                                  shuffle=True)
test_data= test_datagen.flow_from_directory('10_food_classes_all_data/test/',
                                 batch_size=32,
                                 target_size=(128,128),
                                 class_mode='categorical',
                                 seed=42,
                                 shuffle=True)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [94]:
base_model3=tf.keras.applications.ResNet50(include_top=False)
base_model3.trainable=False
inputs= tf.keras.layers.Input(shape=(128,128,3))
x= base_model(inputs)
x= tf.keras.layers.GlobalAveragePooling2D()(x)
x= tf.keras.layers.Flatten()(x)
output= tf.keras.layers.Dense(10, activation='softmax')(x)
model= tf.keras.models.Model(inputs,output)
model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d_1  (None, 512)               0         
 3 (GlobalAveragePooling2D)                                      
                                                                 
 flatten_13 (Flatten)        (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 20029514 (76.41 MB)
Trainable params: 20029514 (76.41 MB)
Non-trainable params: 0 (0.00 Byte)
________________

using resnet ResNet: ResNet (Residual Network) is a popular deep neural network architecture known for its skip connections, which help alleviate the vanishing gradient problem during training. Variants include ResNet-50, ResNet-101, ResNet-152, etc.

In [106]:
model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(train_data, epochs=5,
          validation_data=test_data,
          steps_per_epoch=len(train_data),
          validation_steps=int(0.2*len(test_data)))

Epoch 1/5
235/235 [==============================] - 29s 99ms/step - loss: 1.5605 - accuracy: 0.5192 - val_loss: 1.5559 - val_accuracy: 0.5146
Epoch 2/5
235/235 [==============================] - 21s 90ms/step - loss: 1.4843 - accuracy: 0.5380 - val_loss: 1.6108 - val_accuracy: 0.5104
Epoch 3/5
235/235 [==============================] - 22s 94ms/step - loss: 1.4373 - accuracy: 0.5412 - val_loss: 1.9076 - val_accuracy: 0.4563
Epoch 4/5
235/235 [==============================] - 20s 86ms/step - loss: 1.4261 - accuracy: 0.5468 - val_loss: 1.7072 - val_accuracy: 0.5021
Epoch 5/5
235/235 [==============================] - 21s 90ms/step - loss: 1.3805 - accuracy: 0.5605 - val_loss: 1.6042 - val_accuracy: 0.4833


In [107]:
len(base_model2.layers)

427

In [108]:
for num, layer in enumerate(base_model.layers):
  print(num, layer.name, layer.trainable)

0 input_21 True
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_conv4 True
11 block3_pool True
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True


In [109]:
base_model2.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_31 (InputLayer)       [(None, None, None, 3)]      0         []                            
                                                                                                  
 zero_padding2d_10 (ZeroPad  (None, None, None, 3)        0         ['input_31[0][0]']            
 ding2D)                                                                                          
                                                                                                  
 conv1/conv (Conv2D)         (None, None, None, 64)       9408      ['zero_padding2d_10[0][0]']   
                                                                                                  
 conv1/bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1/conv[0][0]']

In [110]:
base_model.trainable= True
for i ,layer in enumerate(base_model.layers[:-10]):
  layer.trainable=False

opening last 10 layers casue resnet is already preety efficient

In [111]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name, layer.trainable)

0 input_21 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True


In [112]:
model.compile(loss= tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
model.fit(train_data, epochs=10,
          validation_data=test_data,
          steps_per_epoch=len(train_data),
          validation_steps=int(0.2*len(test_data)),
          initial_epoch=5)

Epoch 6/10
235/235 [==============================] - 32s 101ms/step - loss: 1.1381 - accuracy: 0.6303 - val_loss: 1.5796 - val_accuracy: 0.5063
Epoch 7/10
235/235 [==============================] - 21s 91ms/step - loss: 1.1181 - accuracy: 0.6303 - val_loss: 1.5319 - val_accuracy: 0.5396
Epoch 8/10
235/235 [==============================] - 20s 86ms/step - loss: 1.1121 - accuracy: 0.6355 - val_loss: 1.5213 - val_accuracy: 0.5083
Epoch 9/10
235/235 [==============================] - 21s 90ms/step - loss: 1.1040 - accuracy: 0.6352 - val_loss: 1.6030 - val_accuracy: 0.4958
Epoch 10/10
235/235 [==============================] - 20s 83ms/step - loss: 1.1030 - accuracy: 0.6379 - val_loss: 1.5185 - val_accuracy: 0.5250


In [113]:
model.evaluate(test_data)

79/79 [==============================] - 7s 87ms/step - loss: 1.5076 - accuracy: 0.5208


[1.5075868368148804, 0.520799994468689]

my motive for this assignment was not to get 90 percent acccuraccy but to learn about the pretrained models existing in the market and how differently they work with the given dataset.